Parts of this assignment will be **automatically graded**. Please take note of the following:
- Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).
- You can add additional cells, but it is not recommended to (re)move cells. Cells required for autograding cannot be moved and cells containing tests cannot be edited.
- You are allowed to use a service such as [Google Colaboratory](https://colab.research.google.com/) to work together. However, you **cannot** hand in the notebook that was hosted on Google Colaboratory, but you need to copy your answers into the original notebook and verify that it runs succesfully offline. This is because Google Colaboratory destroys the metadata required for grading.
- Name your notebook **exactly** `{TA_name}_{student1_id}_{student2_id}_lab{i}.ipynb`, for example `wouter_12345_67890_lab1.ipynb` (or tim|elise|david|qi, depending on your TA), **otherwise your submission will be skipped by our regex and you will get 0 points** (but no penalty as we cannot parse your student ids ;)).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your names below:

In [ ]:
NAMES = "David Speck, Victor Zuanazzi"

---

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm as _tqdm

import random
import time
from collections import defaultdict

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

EPS = float(np.finfo(np.float32).eps)

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## 1. Temporal Difference (TD) learning (8 points)
Mention one advantage and one disadvantage of Monte Carlo methods. Mention an example where you would prefer to use TD learning.

Advantages of Monte Carlo:

* It is not necessary to know the dynamics of the environment (as is the case for Dynamic Programing), the q and v-values are learned through episodic sampling.
* Monte Carlo is not biased, if compared wth Temporal Difference, it learns q and v-values based on the seen reward for a number of trajectories.

Disadivantages of Monte Carlo:

* Can only be applied for episodic tasks. If termination cannot be guaranteed, then we cannot use Monte Carlo.
* Monte Carlo methods are off policy. It requires us to first collect episodes - for which stochastic policies are required - and later use them for training an target policy. 



TD learning can be applied on or off policy on tasks that are not episodic. TD also allows for on-policy learning.

For the TD algorithms, we will skip the prediction algorithm and go straight for the control setting where we optimize the policy that we are using. In other words: implement SARSA. To keep it dynamic, we will use the windy gridworld environment (Example 6.5).

In [ ]:
from windy_gridworld import WindyGridworldEnv
env = WindyGridworldEnv()

In [ ]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    """
    def policy_fn(observation):
        return int(np.random.rand() * nA) if np.random.rand() < epsilon else np.argmax(Q[observation])
    return policy_fn

In [ ]:
from tqdm import tqdm_notebook as _tqdm # much, much nicer for notebooks!

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

def sarsa(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1, Q=None):
    """
    SARSA algorithm: On-policy TD control. Finds the optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Probability to sample a random action. Float between 0 and 1.
        Q: hot-start the algorithm with a Q value function (optional)
    
    Returns:
        A tuple (Q, stats).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is a list of tuples giving the episode lengths and rewards.
    """
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    if Q is None:
        Q = defaultdict(lambda: np.zeros(env.action_space.n))
        # Q = defaultdict(lambda: np.random.rand(env.action_space.n))  # just for fun!
    
    # Keeps track of useful statistics
    # Where useful stats = episode length and rewards
    stats = []
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)

    for i_episode in tqdm(range(num_episodes)):
        
        # Start the episode from the initial state
        state = env.reset()
        done = False
        i = 0  # Episode duration.
        R = 0  # Reward of the episode
        
        # The agente takes the very first action
        action = policy(state)
        
        while not done:            
            i +=1 
        
            # Transition from the environment
            # One step into the future to collect the reward and the Q(s', a')
            state_new, reward, done, _ = env.step(action)     
            action_new = policy(state_new)
            
            R += reward
            
            # Calculate the new estimative for Q(s,a)
            Q[state][action] = (1 - alpha) * Q[state][action] + alpha*(reward + discount_factor * Q[state_new][action_new])
            
            state = state_new
            action = action_new
            
        stats.append((i, R))
    episode_lengths, episode_returns = zip(*stats)
    return Q, (episode_lengths, episode_returns)

Q_sarsa, (episode_lengths_sarsa, episode_returns_sarsa) = sarsa(env, 1000)

# We will help you with plotting this time
# Thanks =)
plt.plot(episode_lengths_sarsa)
plt.title('Episode lengths SARSA')
plt.show()
plt.plot(episode_returns_sarsa)
plt.title('Episode returns SARSA')
plt.show()

We learn the optimal (non-exploring) policy while using another policy to do exploration, which is where we arrive at _off-policy_ learning. In the simplest variant, we learn our own value by bootstrapping based on the action value corresponding to the best action we could take, while the exploration policy actual follows the $\epsilon$-greedy strategy. This is known as Q-learning.

In [ ]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1, Q=None):
    """
    Q-Learning algorithm: Off-policy TD control. Finds the optimal greedy policy
    while following an epsilon-greedy policy
    
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Probability to sample a random action. Float between 0 and 1.
        Q: hot-start the algorithm with a Q value function (optional)
    
    Returns:
        A tuple (Q, stats).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is a list of tuples giving the episode lengths and rewards.
    """
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    if Q is None:
        Q = defaultdict(lambda: np.zeros(env.action_space.n))
        # Q = defaultdict(lambda: np.random.rand(env.action_space.n))  # just for fun!
    
    # Keeps track of useful statistics
    stats = []
    
    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    

    for i_episode in tqdm(range(num_episodes)):

        # Start the episode from the initial state
        state = env.reset()
        done = False
        i = 0  # Episode duration.
        R = 0  # Reward of the episode
        
        # The agente takes the very first action
        action = policy(state)
        
        while not done:            
            i +=1 
        
            # Transition from the environment
            # One step into the future to collect the reward and the Q(s', a')
            state_new, reward, done, _ = env.step(action)     
            action_new = policy(state_new)
            
            R += reward
            
            # Calculate the new estimative for Q(s,a) taking the greedy action
            Q[state][action] = (1 - alpha) * Q[state][action] + alpha*(reward + discount_factor * np.max(Q[state_new]))
            
            state = state_new
            action = action_new
        
        stats.append((i, R))
    episode_lengths, episode_returns = zip(*stats)
    return Q, (episode_lengths, episode_returns)

Q_q_learning, (episode_lengths_q_learning, episode_returns_q_learning) = q_learning(env, 1000)

# We will help you with plotting this time 
# Thanks =)
plt.plot(episode_lengths_q_learning)
plt.title('Episode lengths Q-learning')
plt.show()
plt.plot(episode_returns_q_learning)
plt.title('Episode returns Q-learning')
plt.show()

Now compare the episode returns while learning for Q-learning and Sarsa (maybe run some more iterations?), by plotting the returns for both algorithms in a single plot, like in the book, Example 6.6. In order to be able to compare them, you may want to zoom in on the y-axis and smooth the returns (e.g. plotting the $n$ episode average instead).

In [ ]:
def sma(array, window=10000):
    """ Calculates the Simple Moving Average of an array."""
    ret = np.cumsum(array, dtype=float)
    ret[window:] = ret[window:] - ret[:-window]
    return ret[window - 1:] / window

# Colects stats:

num_episodes = 1_000_000 # 100_000
Q_sarsa, (episode_lengths_sarsa, episode_returns_sarsa) = sarsa(env, num_episodes)

Q_q_learning, (episode_lengths_q_learning, episode_returns_q_learning) = q_learning(env, num_episodes)

# Smoothes episode lengths
episode_lengths_sarsa_sma = sma(episode_lengths_sarsa)
episode_lengths_q_learning_sma = sma(episode_lengths_q_learning)

# Smoothes episode returns
episode_returns_sarsa_sma = sma(episode_returns_sarsa)
episode_returns_q_learning_sma = sma(episode_returns_q_learning)

# Plot Episode Lengths
plt.plot(episode_lengths_sarsa_sma, label='SARSA')
plt.plot(episode_lengths_q_learning_sma, label='Q')
plt.title('Episode lengths')
plt.legend()
plt.show()

# Plot Episode returns
plt.plot(episode_returns_sarsa_sma, label='SARSA')
plt.plot(episode_returns_q_learning_sma, label='Q-learning')
plt.title('Episode returns')
plt.legend()
plt.show()

Which algorithm achieves higher return during learning? How does this compare to Example 6.6 from the book? Try to explain your observations.

The differences become quite evident when we plot the results smoothed for 100 episodes, 1k episodes were generated per algorithm (Those results also agree with 100k episodes generated per algorithm). Q-learning achieves higher returns than SARSA. As the reward is proportional to the length of the episode, the episodes become shorter under Q-learning.

The results are the oposite as from the Example 6.6 from the book. But the tasks are also fairly different. We are implementing an agent to solve the Windy grid world, there are no extreme rewards in this task, in fact, the only reward signal is the penalty per step. In the Clif walk, there is an very large penalty for falling off the cliff. Thus SARSA and Q-learning learn different policies. Q-learning learns the optimal path, but the behavior policy requires some stochraticity, which makes the agent occasionally fall off the cliff. SARSA can learn the deterministic policy that takes the agent to the goal thourhg the shortest path. However, when the policy is stochastic (as in the example), it actually learns the safest path. That is because SARSA takes into consideration the dynamics of policy selection, as a result, it learns to avoid dangerous states where high penalties can incur. 

After we have learned the policy, we do not care about exploration any more and we may switch to a deterministic (greedy) policy instead. If we evaluate this for both Sarsa and Q-learning (actually, for Q-learning the learned policy is already deterministic), which policy would you expect to perform better? Why?

*** Improve this answer ***

Both should perform about as well. If we use the always chose the greedy action, both should perform the same actions.

Please run the experiments to test your hypothesis (print or plot your results). How many runs do you need to evaluate the policy? Note: without learning, the order of the episodes is not relevant so a normal `plt.plot` may not be the most appropriate choice.

In [ ]:
# Keeps track of useful statistics
stats = []

# Sub-optimal implementation of greedy policy!
agents = {'q': {'policy' : make_epsilon_greedy_policy(Q_q_learning, 0.0, env.action_space.n),
                'episode_length': [],
                'Rewards': [],},
          'sarsa': {'policy' : make_epsilon_greedy_policy(Q_sarsa, 0.0, env.action_space.n),
                    'episode_length': [],
                    'Rewards': [],}}

num_episodes = 100
for i_episode in tqdm(range(num_episodes)):
    for agent in agents:

        # Start the episode from the initial state
        state = env.reset()
        done = False
        i = 0  # Episode duration.
        R = 0  # Reward of the episode
        
        policy = agents[agent]['policy']
        
        action = policy(state)

        while not done:            
            i +=1 

            # Transition from the environment
            # One step into the future to collect the reward and the Q(s', a')
            state_new, reward, done, _ = env.step(action)     
            action_new = policy(state_new)
            
            # Update the reward.
            R += reward

            state = state_new
            action = action_new

    agents[agent]['episode_length'].append(i)
    agents[agent]['Rewards'].append(R)

# Plot Episode Lengths
plt.hist(agents['q']['episode_length'], label='Q-learning')
plt.hist(agents['sarsa']['episode_length'], label='SARSA')
plt.title('Episode lengths')
plt.legend()
plt.show()

# Plot Episode returns
plt.hist(agents['q']['Rewards'], label='Q-learning')
plt.hist(agents['sarsa']['Rewards'], label='SARSA')
plt.title('Episode returns')
plt.legend()
plt.show()

---
## 2. Deep Q-Network (DQN) (10 points)

In [ ]:
import gym
env = gym.envs.make("CartPole-v0")

In [ ]:
# env is a TimeLimit wrapper around an env, so use env.env to look into the env (but otherwise you can forget about this)
??env.env

In [ ]:
import time
# The nice thing about the CARTPOLE is that it has very nice rendering functionality (if you are on a local environment). Let's have a look at an episode
obs = env.reset()
env.render()
done = False
while not done:
    obs, reward, done, _ = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.05)
env.close()  # Close the environment or you will have a lot of render screens soon

Remember from the previous lab, that in order to optimize a policy we need to estimate the Q-values (e.g. estimate the *action* values). In the CartPole problem, our state is current position of the cart, the current velocity of the cart, the current (angular) position of the pole and the (angular) speed of the pole. As these are continuous variables, we have an infinite number of states (ignoring the fact that a digital computer can only represent finitely many states in finite memory).

Can you think of a way in which we can still use a tabular approach? Why would this work and can you think of an example problem where this would not work?

***To be better writen***

We can discretize the state space, as it is bounded. For instance, angular position of the pole is in the range $[0, 2\pi]$, the speed will have similar bounds and so on. This may work well in this particular case be

This approach will not work for highly dimensional states, in order to have fine grained discretation over each dimension we may end up having just too many states to ever learn a value function.

### 2.1 Implement Q-Network

We will not use the tabular approach but approximate the Q-value function by a general approximator function. We will skip the linear case and directly use a two layer Neural Network. We use [PyTorch](https://pytorch.org/) to implement the network, as this will allow us to train it easily later. We can implement a model using `torch.nn.Sequential`, but with PyTorch it is actually very easy to implement the model (e.g. the forward pass) from scratch. Now implement the `QNetwork.forward` function that uses one hidden layer with ReLU activation (no output activation).

In [ ]:
class QNetwork(nn.Module):
    
    def __init__(self, num_hidden=128):
        nn.Module.__init__(self)
        self.l1 = nn.Linear(4, num_hidden)
        self.l2 = nn.Linear(num_hidden, 2)
        
#         # Orthogonal Inicialization
#         # Commented to pass the assertion
#         nn.init.orthogonal_(self.l1.weight)
#         nn.init.orthogonal_(self.l2.weight)

    def forward(self, x):
        # the ugliest way of doing it!
        out = self.l2(F.relu(self.l1(x)))
        return out

# Just for fun!
# class QNetwork(nn.Module):
    
#     def __init__(self, num_hidden=128):
#         nn.Module.__init__(self)
#         self.l1 = nn.Linear(4, num_hidden)
#         self.l2 = nn.Linear(num_hidden, 2)
#         self.l3 = nn.Linear(4, num_hidden)
#         self.l4 = nn.Linear(4, num_hidden)
#         self.l5 = nn.Linear(num_hidden, 4)
        
#         # Orthogonal Inicialization
#         nn.init.orthogonal_(self.l1.weight)
#         nn.init.orthogonal_(self.l2.weight)

#     def forward(self, x):
#         # the ugliest way of doing it!
#         x_ = self.l5(F.relu(self.l4(x)))
#         out = self.l2(F.relu(self.l1(x))+ self.l3(x_))
#         return out

In [ ]:
# Let's instantiate and test if it works
num_hidden = 128
torch.manual_seed(1234)
model = QNetwork(num_hidden)

torch.manual_seed(1234)
test_model = nn.Sequential(
    nn.Linear(4, num_hidden), 
    nn.ReLU(), 
    nn.Linear(num_hidden, 2)
)

x = torch.rand(10, 4)

# If you do not need backpropagation, wrap the computation in the torch.no_grad() context
# This saves time and memory, and PyTorch complaints when converting to numpy
with torch.no_grad():
    assert np.allclose(model(x).numpy(), test_model(x).numpy())

### 2.2 Experience Replay

What could be a problem with doing gradient updates on a sequence of state, action pairs $((s_t, a_t), (s_{t+1}, a_{t+1}) ...)$ observed while interacting with the environment? How will using *experience replay* help to overcome this (potential problem)?

***Make answer nicer!***

The gradient descent algorithm used to train neural nets rely on the hypothesis that the samples in the batch are independent. That is not the case if the batch is a sequence, there are plenty of dependencies and correlations between samples.

Experience replay solves this problem by randomly sampling previously visisted $(s_t, a_t)$ to be added to the SGD batch.


Now implement the `push` function that adds a transition to the replay buffer, and the sample function that returns a batch of samples. It should keep at most the maximum number of transitions. Also implement the `sample` function that samples a (random!) batch of data, for use during training (hint: you can use the function `random.sample`).

In [ ]:
class ReplayMemory:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        
    def pop(self, idx=-1):
        self.memory.pop(-1)

    def push(self, transition):
        self.memory.append(transition)
        
        if len(self.memory) > self.capacity:
            self.pop(0)

    def sample(self, batch_size):
        
        idxs = np.random.randint(len(self.memory), size=batch_size)
        
        return np.array(self.memory)[idxs]

    def __len__(self):
        return len(self.memory)

In [ ]:
capacity = 10
memory = ReplayMemory(capacity)

# Sample a transition
s = env.reset()
a = env.action_space.sample()
s_next, r, done, _ = env.step(a)

# Push a transition
memory.push((s, a, r, s_next, done))

# Sample a batch size of 1
print(memory.sample(1))


### 2.3 $\epsilon$psilon greedy policy

In order to learn a good policy, we need to explore quite a bit initially. As we start to learn a good policy, we want to decrease the exploration. As the amount of exploration using an $\epsilon$-greedy policy is controlled by $\epsilon$, we can define an 'exploration scheme' by writing $\epsilon$ as a function of time. There are many possible schemes, but we will use a simple one: we will start with only exploring (so taking random actions) at iteration 0, and then in 1000 iterations linearly anneal $\epsilon$ such that after 1000 iterations we take random (exploration) actions with 5\% probability (forever, as you never know if the environment will change).

In [ ]:
# def get_epsilon(it):
#     lower_bound = 1000
#     upper_bound = 2000
#     if it < lower_bound:
#         return 1.
#     elif it < upper_bound:
#         return 1.95 - 0.95 * it / lower_bound
#     else:
#         return 0.05

def get_epsilon(it):
    if it > 1000:
        return .05
    return (1 - it * .95 / 1000)

In [ ]:
# So what's an easy way to check?
plt.plot([get_epsilon(it) for it in range(5000)])


Now write a function that takes a state and uses the Q-network to select an ($\epsilon$-greedy) action. It should return a random action with probability epsilon (which we will pass later). Note, you do not need to backpropagate through the model computations, so use `with torch.no_grad():` (see above for example). Unlike numpy, PyTorch has no argmax function, but Google is your friend... Note that to convert a PyTorch tensor with only 1 element (0 dimensional) to a simple python scalar (int or float), you can use the '.item()' function.

In [ ]:
def select_action(model, state, epsilon):
    # this select action can be used for whichever number of actions in a state.
    
    with torch.no_grad():
        # Pytorch can be pretty annoying with its data types!
        state = torch.tensor(state, dtype=torch.float)

        # get the q values for each action given by the model
        q_sa = model.forward(state)

        # random number to decice each action to take
        choice = torch.rand(1).item()

        if choice < epsilon:
            # Uniform sampling
            return int(len(q_sa) * choice/epsilon)
        
        else:
            # Select the greedy action
            a, greedy_act = torch.max(q_sa, 0)
            return greedy_act.item()

# just visualizing it:
s = env.reset()
acts = []
eps =[]
l = 5000
for i in range(l):
    eps.append(get_epsilon(i))
    acts.append(select_action(model, s, eps[-1]))

plt.plot(eps)
plt.scatter([i for i in range(l)], acts)
plt.show()

In [ ]:
s = env.reset()
a = select_action(model, s, 0.05)
assert not torch.is_tensor(a)
print (a)


### 2.4 Training function

Now we will implement the function 'train' that samples a batch from the memory and performs a gradient step using some convenient PyTorch functionality. However, you still need to compute the Q-values for the (state, action) pairs in the experience, as well as their target (e.g. the value they should move towards). What is the target for a Q-learning update? What should be the target if `next_state` is terminal (e.g. `done`)?

For computing the Q-values for the actions, note that the model returns all action values where you are only interested in a single action value. Because of the batch dimension, you can't use simple indexing, but you may want to have a look at [torch.gather](https://pytorch.org/docs/stable/torch.html?highlight=gather#torch.gather) or use [advanced indexing](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.indexing.html) (numpy tutorial but works mostly the same in PyTorch). Note, you should NOT modify the function train. You can view the size of a tensor `x` with `x.size()` (similar to `x.shape` in numpy).

In [ ]:
# def compute_q_val(model, state, action):
#     state = torch.tensor(state, dtype=torch.float)
#     q_sa = model(state)
    
#     # there ought to be a vectorized way of doing it!
#     q_vals = [q_sa[i, action[i]].item() for i in range(len(action))]

#     return torch.tensor(q_vals, dtype=torch.float, requires_grad=True)
    
    
# def compute_target(model, reward, next_state, done, discount_factor):
       
#     # Pytorch has to learn converting between floating points itself!
#     next_state = torch.tensor(next_state, dtype=torch.float)

#     #looks for greedy action
#     q_sa = model.forward(next_state)
#     greedy_value, a = torch.max(q_sa, dim=1)
    
#     not_done = torch.tensor([1 - d for d in done], dtype=torch.float, requires_grad=True)

#     return reward + not_done * discount_factor * greedy_value

def compute_q_val(model, state, action):
    return model(state)[torch.arange(state.shape[0]), action]
    
def compute_target(model, reward, next_state, done, discount_factor):
    return reward + discount_factor * model(next_state).max(dim=1).values * (1 - done).float()


def train(model, memory, optimizer, batch_size, discount_factor):
    # DO NOT MODIFY THIS FUNCTION
    
    # don't learn without some decent experience
    if len(memory) < batch_size:
        return None

    # random transition batch is taken from experience replay memory
    transitions = memory.sample(batch_size)
    
    # transition is a list of 4-tuples, instead we want 4 vectors (as torch.Tensor's)
    state, action, reward, next_state, done = zip(*transitions)
    
    # convert to PyTorch and define types
    state = torch.tensor(state, dtype=torch.float)
    action = torch.tensor(action, dtype=torch.int64)  # Need 64 bit to use them as index
    next_state = torch.tensor(next_state, dtype=torch.float)
    reward = torch.tensor(reward, dtype=torch.float)
    done = torch.tensor(done, dtype=torch.uint8)  # Boolean
    
    # compute the q value
    q_val = compute_q_val(model, state, action)
    
    with torch.no_grad():  # Don't compute gradient info for the target (semi-gradient)
        target = compute_target(model, reward, next_state, done, discount_factor)
    
    # loss is measured from error between current and newly expected Q values
    loss = F.smooth_l1_loss(q_val, target)
    

    # backpropagation of loss to Neural Network (PyTorch magic)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()  # Returns a Python scalar, and releases history (similar to .detach())

In [ ]:
# You may want to test your functions individually, but after you do so lets see if the method train works.
batch_size = 64
discount_factor = 0.8
learn_rate = 1e-3
# Simple gradient descent may take long, so we will use Adam
optimizer = optim.Adam(model.parameters(), learn_rate)

# We need a larger memory, fill with dummy data
transition = memory.sample(1)[0]
memory = ReplayMemory(10 * batch_size)
for i in range(batch_size):
    memory.push(transition)

# Now let's see if it works
loss = train(model, memory, optimizer, batch_size, discount_factor)

print (loss)


### 2.5 Put it all together

Now that you have implemented the training step, you should be able to put everything together. Implement the function `run_episodes` that runs a number of episodes of DQN training. It should return the durations (e.g. number of steps) of each episode. Note: we pass the train function as an argument such that we can swap it for a different training step later.

In [ ]:
def run_episodes(train, model, memory, env, num_episodes, batch_size, discount_factor, learn_rate,render=False):
    
    optimizer = optim.Adam(model.parameters(), learn_rate)
    
    global_steps = 0  # Count the steps (do not reset at episode start, to compute epsilon)
    episode_durations = [] 
    losses = []
    
    max_duration = 1000
    repeat_train = 2
    
    global_iter = 0
    for i in tqdm(range(num_episodes)):
        
        # start episode
        state = env.reset()
        if render and not(i%10):
            env.close()  # Close the environment of the last run
            env.render()
        
        duration = 0
        loss = 0
        done = False
        while not done:
            duration += 1
            global_iter += 1

            # computes one step
            epsilon = get_epsilon(global_iter)
            action = select_action(model, state, epsilon)
            s_next, reward, done, _ = env.step(action)
            if render and not(i%10):
                env.render()
                #time.sleep(0.01)

            # Push a transition
            memory.push((state, action, reward, s_next, done))
            
            # train the model
            if len(memory) > 2 * batch_size:
                for j in range(repeat_train):
                    # just something to see if things get better
                    loss += train(model, memory, optimizer, batch_size, discount_factor)

            state = s_next
            
            if duration > max_duration:
                break
        
        episode_durations.append(duration)
        
        # Heuristic that works nicely =)
        # We start repeating the training as soon as the duration increases significantly
        # It takes a while though...
        repeat_train = int(duration / episode_durations[0])**2 + 1

    if render:
        env.close()  # Close the environment or you will have a lot of render screens soon
     
    return episode_durations

In [ ]:
# For debugin purposes
num_episodes = 100
batch_size = 64
discount_factor = 0.8
learn_rate = 1e-3
memory = ReplayMemory(10000)
num_hidden = 128
seed = 42  # This is not randomly chose

# We will seed the algorithm (before initializing QNetwork!) for reproducability
random.seed(seed)
torch.manual_seed(seed)
env.seed(seed)

model = QNetwork(num_hidden)

episode_durations = run_episodes(train, model, memory, env, num_episodes, batch_size, discount_factor, learn_rate, render=True)

# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

plt.plot(smooth(episode_durations, 10))
plt.title('Episode durations per episode')
plt.show()


In [ ]:
# Let's run it!
num_episodes = 100
batch_size = 64
discount_factor = 0.8
learn_rate = 1e-3
memory = ReplayMemory(10000)
num_hidden = 128
seed = 42  # This is not randomly chosen

# We will seed the algorithm (before initializing QNetwork!) for reproducability
random.seed(seed)
torch.manual_seed(seed)
env.seed(seed)

model = QNetwork(num_hidden)

episode_durations = run_episodes(train, model, memory, env, num_episodes, batch_size, discount_factor, learn_rate)

In [ ]:
# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

plt.plot(smooth(episode_durations, 10))
plt.title('Episode durations per episode')


---

## 3. Policy Gradient (8 points)

So we have spent a lot of time working on *value based* methods. We will now switch to *policy based* methods, i.e. learn a policy directly rather than learn a value function from which the policy follows. Mention two advantages of using a policy based method.

*** Improve answer ***

We cannot find the optimal stochastic policy by learning Q and V function approximations. By learning the policy directly we can find the optimal.

For many problems, the policy function is simpler to approximate than the Q and V functions.

It is much easier to integrate prior knowledge of the desired policy when we are learning it directly.

### 3.1 Policy Network

In order to do so, we will implement a Policy network. Although in general this does not have to be the case, we will use an architecture very similar to the Q-network (two layers with ReLU activation for the hidden layer). Since we have discrete actions, our model will output one value per action, where each value represents the (normalized!) log-probability of selecting that action. *Use the (log-)softmax activation function.*

In [ ]:
class PolicyNetwork(nn.Module):
    
    def __init__(self, num_hidden=128):
        nn.Module.__init__(self)
        self.l1 = nn.Linear(4, num_hidden)
        self.l2 = nn.Linear(num_hidden, 2)

    def forward(self, x):
        
        x_ = F.relu(self.l1(x))
        out = nn.LogSoftmax()(self.l2(x_))
        
        return out

In [ ]:
# Let's instantiate and test if it works
num_hidden = 128
torch.manual_seed(1234)
model = PolicyNetwork(num_hidden)

x = torch.rand(10, 4)

log_p = model(x)

# Does the outcome make sense?
print(log_p.exp())


### 3.2 Monte Carlo REINFORCE

Now we will implement the *Monte Carlo* policy gradient algorithm. Remember from lab 1 that this means that we will estimate returns for states by sample episodes. Compared to DQN, this means that we do *not* perform an update step at every environment step, but only at the end of each episode. This means that we should generate an episode of data, compute the REINFORCE loss (which requires computing the returns) and then perform a gradient step.

To help you, we already implemented a few functions that you can (but do not have to) use.

* You can use `torch.multinomial` to sample from a categorical distribution.
* The REINFORCE loss is defined as $- \sum_t \log \pi_\theta(a_t|s_t) G_t$, which means that you should compute the (discounted) return $G_t$ for all $t$. Make sure that you do this in **linear time**, otherwise your algorithm will be very slow! Note the - (minus) since you want to maximize return while you want to minimize the loss.
* Importantly, you should **normalize the returns** (not the rewards!, e.g. subtract mean and divide by standard deviation within the episode) before computing the loss, or your estimator will have very high variance.

In [ ]:
def select_action(model, state):
    # Samples an action according to the probability distribution induced by the model
    # Also returns the log_probability
    log_p = model(state)
    action = torch.multinomial(log_p.exp(), num_samples=1)
    return action.item(), log_p[action]

def run_episode(env, model, render=True):
    
    # start episode
    episode = []

    state = env.reset()
    if render:
        env.render()
    
    max_duration = 1000

    done = False
    duration = 0
    while not done:
        duration += 1

        # computes one step
        state = torch.tensor(state, dtype=torch.float)
        action, log_p_action = select_action(model, state)
        s_next, reward, done, _ = env.step(action)
        
        episode.append((state, action, log_p_action, reward))
        
        if render:
            env.render()
        
        state = s_next
        
        if duration > max_duration:
            break
    
    env.close()
    
    return episode

def compute_reinforce_loss(episode, discount_factor):
    # Compute the reinforce loss
    # Make sure that your function runs in LINEAR TIME
    # Don't forget to normalize your RETURNS (not rewards)
    # Note that the rewards/returns should be maximized 
    # while the loss should be minimized so you need a - somewhere
    
    # From the end to the begining of the episode:
    # a) log_p stores the log_probability given by the NN,
    # b) discounted_r stores the G at each time step.
    _, _, l_p, reward = episode[-1]
    discounted_r = [reward]
    log_p = [l_p]
    loss = 0
    
    for _, _, l_p, reward in reversed(episode[:-1]):
        discounted_r.append(discount_factor * discounted_r[-1] + reward)
        log_p.append(l_p)
    
    # Nomalize the return
    discounted_r = torch.tensor(discounted_r)
    mean = discounted_r.mean()
    var = discounted_r.var() 
    if var == 0:
        print(f"var problem: var = {var}, normalization will take var = 1 instead.")
        var = 1
        
    discounted_r = (torch.tensor(discounted_r) - mean)/ var
    
    for l_p, reward in zip(log_p, discounted_r):
        loss -= l_p * reward
    
    return loss

def run_episodes_policy_gradient(model, env, num_episodes, discount_factor, learn_rate):
    
    optimizer = optim.Adam(model.parameters(), learn_rate)
    
    episode_durations = []
    max_ep = 0
    for i in range(num_episodes):
        
        episode = run_episode(env, model, render=False)
        
        loss = compute_reinforce_loss(episode, discount_factor)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        episode_durations.append(len(episode))
#         if max_ep < len(episode):
#             print("{2} Episode {0} finished after {1} steps"
#                   .format(i, len(episode), '\033[92m' if len(episode) >= 195 else '\033[99m'))
#             max_ep = len(episode)
                           
        if not (i % (int(num_episodes/10))):
            print("{2} Episode {0} finished after {1} steps"
                  .format(i, len(episode), '\033[92m' if len(episode) >= 195 else '\033[99m'))
        episode_durations.append(len(episode))
        
    return episode_durations

In [ ]:
# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [ ]:
# Feel free to play around with the parameters!
num_episodes = 200
discount_factor = 0.99
learn_rate = 0.01
seed = 42
random.seed(seed)
torch.manual_seed(seed)
env.seed(seed)

model = PolicyNetwork(num_hidden)

episode_durations_policy_gradient = run_episodes_policy_gradient(
    model, env, num_episodes, discount_factor, learn_rate)

plt.plot(smooth(episode_durations_policy_gradient, 10))
plt.title('Episode durations per episode')
plt.legend(['Policy gradient'])

## 4. Deep Reinforcement Learning (5 bonus points)
Note that so far we used the state variables as input. However, the true power of Deep Learning is that we can directly learn from raw inputs, e.g. we can learn to balance the cart pole *by just looking at the screen*. This probably means that you need a deep(er) (convolutional) network, as well as tweaking some parameters, running for more iterations (perhaps on GPU) and do other tricks to stabilize learning. Can you get this to work? This will earn you bonus points!

Hints:
* You may want to use [Google Colab](https://colab.research.google.com/) such that you can benefit from GPU acceleration.
* Even if you don't use Colab, save the weights of your final model and load it in the code here (see example below). Hand in the model file with the .ipynb in a .zip. We likely won't be able to run your training code during grading!
* Preprocessing is already done for you, and the observation is the difference between two consequtive frames such that the model can 'see' (angular) speed from a single image. Now do you see why we (sometimes) use the word observation (and not state)?

In [ ]:
import torchvision.transforms as T
from PIL import Image


resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])

class CartPoleRawEnv(gym.Env):
    
    def __init__(self, *args, **kwargs):
        self._env = gym.make('CartPole-v0', *args, **kwargs)  #.unwrapped
        self.action_space = self._env.action_space
        screen_height, screen_width = 40, 80  # TODO
        self.observation_space = gym.spaces.Box(
            low=0, high=255, 
            shape=(screen_height, screen_width, 3), dtype=np.uint8)
    
    def seed(self, seed=None):
        return self._env.seed(seed)
    
    def reset(self):
        s = self._env.reset()
        self.prev_screen = self.screen = self.get_screen()
        return self._get_observation()
    
    def step(self, action):
        s, r, done, info = self._env.step(action)
        self.prev_screen = self.screen
        self.screen = self.get_screen()
        return self._get_observation(), r, done, info
    
    def _get_observation(self):
        return self.screen - self.prev_screen
    
    def _get_cart_location(self, screen_width):
        _env = self._env.unwrapped
        world_width = _env.x_threshold * 2
        scale = screen_width / world_width
        return int(_env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

    def get_screen(self):
        screen = self._env.unwrapped.render(mode='rgb_array').transpose(
            (2, 0, 1))  # transpose into torch order (CHW)
        # Strip off the top and bottom of the screen
        _, screen_height, screen_width = screen.shape
        screen = screen[:, screen_height * 4 // 10:screen_height * 8 // 10]
        view_width = screen_height * 8 // 10
        cart_location = self._get_cart_location(screen_width)
        if cart_location < view_width // 2:
            slice_range = slice(view_width)
        elif cart_location > (screen_width - view_width // 2):
            slice_range = slice(-view_width, None)
        else:
            slice_range = slice(cart_location - view_width // 2,
                                cart_location + view_width // 2)
        # Strip off the edges, so that we have a square image centered on a cart
        screen = screen[:, :, slice_range]
        # Convert to float, rescare, convert to torch tensor
        # (this doesn't require a copy)
        screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
        screen = torch.from_numpy(screen)
        # Resize, and add a batch dimension (BCHW)
        #return screen.unsqueeze(0).to(device)
        return resize(screen).unsqueeze(0)
    
    def close(self):
        return self._env.close()

raw_env = CartPoleRawEnv()
s = raw_env.reset()

# 
s, r, done, _ = raw_env.step(env.action_space.sample())

raw_env.reset()
plt.figure()
plt.imshow(raw_env.get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()

# Observations are (-1, 1) while we need to plot (0, 1) so show (rgb + 1) / 2
plt.figure()
plt.imshow((s.cpu().squeeze(0).permute(1, 2, 0).numpy() + 1) / 2,
           interpolation='none')
plt.title('Example observation')
plt.show()
raw_env.close()

In [ ]:
# Maybe you should make it a bit deeper?
class DeepPolicy(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.l1 = nn.Linear(40 * 80 * 3, 2)

    def forward(self, x):
        # Flatten
        return F.log_softmax(self.l1(x.view(x.size(0), -1)), -1)
    
policy = DeepPolicy()
filename = 'weights.pt'

if os.path.isfile(filename):
    print(f"Loading weights from {filename}")
    weights = torch.load(filename, map_location='cpu')
    
    policy.load_state_dict(weights['policy'])
    
else:
    # Train
    
    ### TODO some training here, maybe? Or run this on a different machine?
    torch.manual_seed(42)
    
    print(f"Saving weights to {filename}")
    torch.save({
        # You can add more here if you need, e.g. critic
        'policy': policy.state_dict()  # Always save weights rather than objects
    },
    filename)
    
def bonus_get_action(x):
    return policy(x).exp().multinomial(1)[:, 0]

In [ ]:
seed = 42
episode_durations = []
for i in range(20):  # Not too many since it may take forever to render
    test_env = CartPoleRawEnv()
    test_env.seed(seed + i)
    state = test_env.reset()
    done = False
    steps = 0
    while not done:
        steps += 1
        with torch.no_grad():
            action = bonus_get_action(state).item()
        state, reward, done, _ = test_env.step(action)
    episode_durations.append(steps)
    test_env.close()
    
plt.plot(episode_durations)
plt.title('Episode durations')
plt.show()
